<a href="https://colab.research.google.com/github/CheesyPitsa/MainProject/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2    
import math   
import pandas as pd 
import numpy as np    
from keras.utils import np_utils
from skimage.transform import resize   
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint
import csv
from tensorflow.keras.utils import get_file
from urllib.request import urlopen

In [ ]:
train_data = pd.DataFrame()
train_image = []
train_class = []
for i in range(46):
    train_image.append('nude' + str(i))
    train_class.append('nude')
for i in range(50):
    train_image.append('safe' + str(i))
    train_class.append('safe')
for i in range(50):
    train_image.append('sexy' + str(i))
    train_class.append('sexy')
train_data['image'] = train_image
train_data['class'] = train_class
train_data.to_csv('new1.txt',header=True, index=False)
train_data.head()

,image,class
0,nude0,nude
1,nude1,nude
2,nude2,nude
3,nude3,nude
4,nude4,nude


In [ ]:
train_data['class'].drop_duplicates()

0     nude
46    safe
96    sexy
Name: class, dtype: object

In [ ]:
train = pd.read_csv('new1.txt')
train.shape

(146, 2)

In [ ]:
train_image = []
j=0

with open("new1.txt") as file:
  array = [row.strip() for row in file]
  for i in tqdm(range(train.shape[0])):
    img = image.load_img(get_file(train['image'][i], array[j]), target_size=(80,80,3))
    j=j+1
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

X = np.array(train_image)
X.shape

100%|██████████| 146/146 [00:04<00:00, 33.61it/s]


(146, 80, 80, 3)

In [ ]:
a = np.random.uniform(400, 800, size = (3, 128, 128, 3))
print(a.size * a.itemsize)

1179648


In [ ]:
y = train['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [ ]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
y_train

,nude,safe,sexy
40,1,0,0
133,0,0,1
78,0,1,0
0,1,0,0
25,1,0,0
...,...,...,...
6,1,0,0
23,1,0,0
11,1,0,0
66,0,1,0


In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(80,80,3))

In [ ]:
X_train = base_model.predict(X_train)
X_train.shape

4/4 [==============================] - 0s 28ms/step


(116, 2, 2, 512)

In [ ]:
X_test = base_model.predict(X_test)
X_test.shape

1/1 [==============================] - 0s 20ms/step


(30, 2, 2, 512)

In [ ]:
X_train = X_train.reshape(116, 2*2*512)
X_test = X_test.reshape(30, 2*2*512)

In [ ]:
max = X_train.max()
print(max)
X_train = X_train/max
X_test = X_test/max

10.747984


In [ ]:
X_train.shape

(116, 2048)

In [ ]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(2048,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
mcp_save = ModelCheckpoint('weight_80_36000.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
y_train = y_train.to_numpy(copy=False)
y_test = y_test.to_numpy(copy=False)

In [ ]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

In [ ]:
predictions = model.predict(X_test)

1/1 [==============================] - 0s 18ms/step


In [ ]:
im = 6
np.argmax(predictions[im])
print (predictions [im])

[4.0311557e-03 9.9596763e-01 1.1965305e-06]


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 1024)              2098176   
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              